In [1]:
## fill nas with median
## very rude transformation of dates
## test/train split
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import missingno as msno
import pickle
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from scipy import stats
from scipy.stats import zscore
# from fancyimpute import *

%matplotlib inline

In [2]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)

def batch_save(train_x, train_y, valid_x, valid_y, test, postfix):
    train_x.reset_index().to_feather("tmp/train_x_{}".format(postfix))
    train_y.reset_index().to_feather("tmp/train_y_{}".format(postfix))
    valid_x.reset_index().to_feather("tmp/valid_x_{}".format(postfix))
    valid_y.reset_index().to_feather("tmp/valid_y_{}".format(postfix))
    test.reset_index().to_feather("tmp/test_{}".format(postfix))
    
def batch_load(postfix):
    train_x = pd.read_feather("tmp/train_x_{}".format(postfix))
    train_y = pd.read_feather("tmp/train_y_{}".format(postfix))
    valid_x = pd.read_feather("tmp/valid_x_{}".format(postfix))
    valid_y = pd.read_feather("tmp/valid_y_{}".format(postfix))
    return train_x, train_y, valid_x, valid_y

def my_roc(y_true, y_prob):
    if isinstance(y_true,pd.core.series.Series):
        y_true = np.array(y_true.tolist())
    if isinstance(y_true,list):
        y_true = np.array(y_true)
    sort_index = np.argsort(y_prob)[::-1]
    y_prob = y_prob[sort_index]
    y_true = y_true[sort_index]
    num_p = y_true.sum()
    num_n = len(y_true) - num_p
    fp = 0
    tp = 0
    fps = []
    tps = []
    prob_prev = -99
    i = 0
    while i < len(y_true):
        if y_prob[i]!=prob_prev:
            fps.append(fp/num_n)
            tps.append(tp/num_p)
            prob_prev=y_prob[i]
        if y_true[i]==1:
            tp+=1
        else:
            fp+=1
        i+=1
    fps.append(fp/num_n)
    tps.append(tp/num_p)
    return np.array(fps), np.array(tps)

def my_score3(predictions, xtrain): ##Adapted from SKlearn, conservative (actual should be higher)
    ground_truth = y_true = xtrain.get_label()
    fpr,tpr = my_roc(ground_truth, predictions)
#     plt.scatter(fpr, tpr)
#     plt.show()
    tpr1 = tpr[(fpr>=0.001).argmax()-1]
    tpr2 = tpr[(fpr>=0.005).argmax()-1] 
    tpr3 = tpr[(fpr>=0.01).argmax()-1]
    return 'score', 0.4 * tpr1 + 0.3 * tpr2 + 0.3* tpr3

def norm_standardize(df, start=0):
    for col in df.columns[start:]:
#         avg = df[col].mean()
#         std = df[col].std(ddof=0)
#         if std != 0:
#             df[col] = (df[col]-avg)/std
#         else:
#             print(col)
        a = df[col]
        z = a
        z[~np.isnan(a)] = zscore(a[~np.isnan(a)])
        df[col] = z
            
def norm_maxmin(df, start=0):
    for col in df.columns[start:]:
        df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())

In [3]:
dtype = load_obj('dict_dtype')

my_dict = load_obj('my_dict')

**** Load the training data and test data ****

In [ ]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)

test = pd.read_csv("atec_anti_fraud_test_a.csv",parse_dates=['date'], dtype = dtype)

In [ ]:
df_missing_ratio = load_obj('df_missing_ratio')
# display_all(df_missing_ratio)

selected_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']<0.1].index.tolist()]
all_nan_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']>0.9].index.tolist()]

# # use the columns with no or few missing values
# data = data.drop(all_nan_cols, axis=1)
# test = test.drop(all_nan_cols, axis=1)

**** Perform one hot encoding for the columns with no more than 10 unique values ****

In [ ]:
# one hot encoding for the columns with no more than 10 unique values
for col in data.columns[3:]:
    data_unique = data[col].unique()
    test_unique = test[col].unique()
    if data_unique[~np.isnan(data_unique)].min() == test_unique[~np.isnan(test_unique)].min() and \
    data_unique[~np.isnan(data_unique)].max() == test_unique[~np.isnan(test_unique)].max() and \
    data_unique.shape[0] == test_unique.shape[0] and \
    data_unique.shape[0] <= 10:
        data[col].fillna(-1.0)
        test[col].fillna(-1.0)
        for num in data[col].unique():
            new_col = '{}={}'.format(col, num)
            data[new_col] = data[col].apply(lambda x: np.isnan(x) if np.isnan(num) else x==num)
            test[new_col] = test[col].apply(lambda x: np.isnan(x) if np.isnan(num) else x==num)
            data.drop([col], axis=1)
            test.drop([col], axis=1)

**** save the file locally to save the time of preprocessing ****

In [ ]:
save_obj(data, 'train_onehot')
save_obj(test, 'test_onehot')

**** load the onehot encoding preprocessed dataset ****

In [37]:
# load the training set and test set
data = load_obj('train_onehot')
test = load_obj('test_onehot')

**** Perform normalization ****

In [ ]:
# normalization with maxmin
norm_maxmin(data, 3)
norm_maxmin(test, 2)

In [ ]:
# normalization with zscore
norm_standardize(data, 3)
norm_standardize(test, 2)

**** Sort the training data and remove the unlabeled data ****

In [38]:
# temporially ignore the rows without labels
data.sort_values('date',inplace=True)
unlabeled = data[data['label']==-1]

In [44]:
# with the prediction on the unlabeled training data, set the labels for unlabeled data
data.loc[data['label']==-1,'label'] = pd.Series((pred_xgunlabeled>0.5).astype(int), name='label', index=data.loc[data['label']==-1,'label'].index)

In [ ]:
data = data[data['label']!=-1]

**** Create validation set and training set ****

In [46]:
train = data.iloc[:len(data) * 9 // 10]
valid = data.iloc[len(data) * 9 // 10:]

In [47]:
train_y = train[['label']]
train_x = train.iloc[:,3:]
valid_y = valid[['label']]
valid_x = valid.iloc[:,3:]

In [48]:
xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2')]

**** Train the model ****

In [49]:
# set up the parameters
params = {'max_depth': 8, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['auc', "error"]
params["scale_pos_weight"] = 5
num_rounds = 5
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 10

In [50]:
%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist, feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

[0]	train-auc:0.996422	train-error:0.008338	val_1-auc:0.986098	val_1-error:0.013431	val_2-auc:0.987777	val_2-error:0.010877	train-score:0.72634	val_1-score:0.488484	val_2-score:0.593691
Multiple eval metrics have been passed: 'val_2-score' will be used for early stopping.

Will train until val_2-score hasn't improved in 1000 rounds.
[1]	train-auc:0.996477	train-error:0.008324	val_1-auc:0.98604	val_1-error:0.013551	val_2-auc:0.987668	val_2-error:0.010938	train-score:0.72928	val_1-score:0.485131	val_2-score:0.594497
[2]	train-auc:0.996526	train-error:0.008316	val_1-auc:0.986004	val_1-error:0.013672	val_2-auc:0.987639	val_2-error:0.010877	train-score:0.731483	val_1-score:0.48484	val_2-score:0.595705
[3]	train-auc:0.996592	train-error:0.008316	val_1-auc:0.986014	val_1-error:0.013692	val_2-auc:0.987665	val_2-error:0.010958	train-score:0.732333	val_1-score:0.482507	val_2-score:0.593557
[4]	train-auc:0.996629	train-error:0.008304	val_1-auc:0.98595	val_1-error:0.013712	val_2-auc:0.987726	val_2

**** save or load the model ****

In [ ]:
# save the model
xgb_model.save_model('model_log/0012.model')
# dump model with feature map
xgb_model.dump_model('model_log/dumpraw0012.txt')

In [27]:
# load the model
xgb_model = xgb.Booster({'nthread': 4})  # init model
xgb_model.load_model('model_log/0011.model')  # load model

**** train the existing model on the validation set ****

In [ ]:
xgval = xgb.DMatrix(valid_x.values, label=valid_y.values)

In [ ]:
%time xgb_model = xgb.train(params, xgval, num_rounds, [(xgval, 'validation')], feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

**** predict on the testset ****

In [ ]:
# load test data
test_whole = pd.read_feather("tmp/test_native")

In [ ]:
xgtest = xgb.DMatrix(test.iloc[:,2:].values)

# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)

res = pd.concat([test.id, pd.Series(list(preds), name='score')], axis=1)

res.to_csv("0012.csv", index=False)

In [ ]:
# the positive ratio of the test data
print('Ratio of positive label in unlabeled data: {}%'.format((preds>0.5).sum()/preds.shape[0]*100))

**** predict on the unlabeled training set ****

In [39]:
xgunlabeled = xgb.DMatrix(unlabeled.iloc[:,3:].values)

In [40]:
unlabeled.reset_index(drop=True, inplace=True)

In [41]:
pred_xgunlabeled = xgb_model.predict(xgunlabeled)

In [57]:
# set the labels for the unlabeled training data
unlabeled['label'] = pd.Series((pred_xgunlabeled>0.5).astype(int), name='label')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
res = pd.concat([unlabeled.id, pd.Series(list(pred_xgunlabeled), name='score')], axis=1)
res.to_csv("Yabin_unlabeled0011.csv", index=False)

In [ ]:
# the positive ratio of the unlabeled data
print('Ratio of positive label in unlabeled data: {}%'.format((pred_xgunlabeled>0.5).sum()/res.shape[0]*100))

In [56]:
unlabeled

,id,label,date,f1,f2,f3,f4,f5,f6,f7,...,f168=nan,f168=3.0,f168=4.0,f169=2.0,f169=1.0,f169=nan,f169=3.0,f169=4.0,f169=5.0,f169=6.0
0,3c4b3baa09510eec473f45079c7c7977fc01fdfed93a8b...,0,2017-09-05,0,1,1,1,100702.0,0,3,...,False,False,False,False,True,False,False,False,False,False
1,643b560780177054c375bd2ddd8a1ac6b99b728f016c75...,1,2017-09-05,1,1,1,0,100803.0,0,3,...,False,False,False,True,False,False,False,False,False,False
2,addd09c73d0bfa3d98e252865c3343b534db2f0701e089...,0,2017-09-05,1,1,1,0,100803.0,0,4,...,False,False,False,False,True,False,False,False,False,False
3,f8e66a38649c631bc205bd9f5cb953e214b17a3d9c80f2...,0,2017-09-05,1,1,1,0,100802.0,0,6,...,True,False,False,False,False,True,False,False,False,False
4,61dacd672d0d12112ae4c5a9f0aa7d348c9566ba60bda3...,0,2017-09-05,1,0,0,1,100803.0,0,5,...,False,False,False,False,True,False,False,False,False,False
5,bbd743c945e39779d7136f90dcd6e005b04d6660dd65ec...,1,2017-09-05,2,1,1,1,NaN,3,2,...,True,False,False,False,False,True,False,False,False,False
6,072a5f1007ec94ff4e6547783ec3dff4c72073dcfe5dfa...,0,2017-09-05,0,1,0,0,100803.0,0,5,...,False,False,False,False,True,False,False,False,False,False
7,3842de26f717890858813543c6938c0fdb346b3e3fc715...,0,2017-09-05,0,0,0,1,100803.0,1,4,...,False,False,False,False,True,False,False,False,False,False
8,1ac41ee380516d13d2692b74eb08e7b33135fe8b745ca8...,0,2017-09-05,0,0,0,1,100803.0,1,4,...,False,False,False,True,False,False,False,False,False,False
9,1e06c933127058d1d88c63084522d4aa19dd0d55bc61ee...,1,2017-09-05,1,0,0,1,100804.0,0,2,...,False,False,False,False,True,False,False,False,False,False
